In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import openpyxl
import bs4


In [15]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option('detach',True)

In [17]:
driver = webdriver.Chrome(options=options)
actions=ActionChains(driver)
driver.implicitly_wait(5)

In [18]:
driver.get("https://www.amazon.com/")



In [19]:


search_box = driver.find_element(By.ID, "twotabsearchtextbox")

# Type your search query
search_query = "laptop"
search_box.send_keys(search_query)

# Press Enter or click the search button
search_box.send_keys(Keys.RETURN)

sleep(2)

In [214]:
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [5]:
if __name__ == '__main__':
    
    current_url = driver.current_url
    a=current_url
    print(a)
    #webpage= driver.page_source
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    webpage = requests.get(a,headers=HEADERS)
    #html = driver.page_source
    soup=bs4.BeautifulSoup(webpage.content,"lxml")
    links = soup.find_all("a", attrs={'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        if link.startswith("http"):
            full_url = link
        else:
            full_url = "https://www.amazon.com" + link
        new_webpage = requests.get(full_url, headers=HEADERS)


        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)
    
    amazon_df=pd.DataFrame(d)
    amazon_df.to_excel("d.xlsx")

    amazon_df=pd.DataFrame(d)
    amazon_df.to_excel("d.csv")

NameError: name 'driver' is not defined

In [217]:
amazon_df

,title,price,rating,reviews,availability
0,Amazon Fire Max 11 tablet (newest model) vivid...,,4.5 out of 5 stars,"13,230 ratings",Not Available
1,"2025 Laptop, Laptop Computer with Intel Quad-C...",,4.6 out of 5 stars,10 ratings,Not Available
2,"SGIN 15.6 Inch Laptop Computer, Windows 11 Lap...",,4.5 out of 5 stars,44 ratings,Not Available
3,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",,4.1 out of 5 stars,"3,265 ratings",Not Available
4,"ACEMAGIC 16.0-inch Gaming Laptop, 2025 16GB DD...",,4.8 out of 5 stars,19 ratings,Not Available
5,"15.6 Inch Laptop with Office 365, 4GB RAM, 128...",,4.4 out of 5 stars,164 ratings,Not Available
6,Laptop Computer Windows Laptop Intel N97 15.6-...,,4.8 out of 5 stars,15 ratings,Not Available
7,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,$299.99,4.2 out of 5 stars,"4,554 ratings",In Stock
8,Apple 2025 MacBook Air 13-inch Laptop with M4 ...,,4.7 out of 5 stars,789 ratings,Not Available
9,"Naclud 15.6 Inch Laptop Computer, Windows 11 L...",,4.4 out of 5 stars,39 ratings,Not Available
